In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualization_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os
import random

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix, Sphere, Rgba
import pydrake.symbolic as sym
import time
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions
from pydrake.all import RationalForwardKinematics, CspaceFreeLine
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath
from pydrake.polynomial import Polynomial as PolynomialCommon

In [4]:
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions
import logging
drake_logger = logging.getLogger("drake")
# drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)




pendulum_world = FindResourceOrThrow("drake/C_Iris_Examples/assets/double_pendulum_world.urdf")

models = []
models.append(parser.AddModelFromFile(pendulum_world))

plant.Finalize()
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())



# The object we will use to perform our certification.
t0 = time.time()
cspace_free_path = CspaceFreePath(plant, scene_graph, q_star, maximum_path_degree = 1, plane_order = 1)
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity if you want
# to see the plant with the collision geometries.
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, cspace_free_path, viz_role=Role.kIllustration)
visualizer.visualize_collision_constraint(factor = 1.2, num_points = 100)
visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat_task_space.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,1,0])))

INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Meshcat listening for connections at http://localhost:7001


Time to construct line certifier = 0.004147529602050781s


In [6]:
# compute limits in t-space
limits_q = [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]
do_viz = True
# compute limits in s-space
limits_s = []
for q in [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star))
limits_s = np.array(limits_s)

In [7]:
# visualizer.meshcat_cspace.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
# visualizer.meshcat_task_space.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,1,0])))

In [8]:
# # filter fused joints self collisions so they don't interfere with collision engine
# digaram = visualizer.diagram
# context = visualizer.diagram_context
# sg_context = scene_graph.GetMyContextFromRoot(context)
# inspector = scene_graph.model_inspector()

# pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

# gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
# get_name_of_gid = lambda gid : inspector.GetName(gid)
# gids.sort(key=get_name_of_gid)

# id_pairs_of_interest = [(gid1, gid2) for i, gid1 in enumerate(gids) for gid2 in gids[i+1:] ]
# print(id_pairs_of_interest)
# visualizer.collision_pairs_of_interest = id_pairs_of_interest


## Set up the sliders so we can move the plant around manually

In [9]:
sliders = []

for i, q in enumerate(plant.GetPositionLowerLimits()):
    sliders.append(widgets.FloatSlider(min=q,
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=0, description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.array([0,0])
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_q(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [10]:
s0 = np.array([0,0])
s1_safe = np.array([-0.25, 0.9])
s1_unsafe = np.array([1,1])

def plot_edge(s0, s1, s0_name = "s0", s1_name = "s1"):
    visualizer.meshcat_cspace.SetObject(f"/{s0_name}",
                                 Sphere(0.05),
                                 Rgba(0.0, 1.0, 0, 1))
    s0_3d = np.array([s0[0],s0[1],0])
    s1_3d = np.array([s1[0],s1[1],0])
    print(s0_3d)
    print(s1_3d)
    visualizer.meshcat_cspace.SetTransform(f"/{s0_name}",
                                  RigidTransform(RotationMatrix(),
                                                 s0_3d))
    
    visualizer.meshcat_cspace.SetObject(f"/{s1_name}",
                                 Sphere(0.05),
                                 Rgba(1.0, 0.0, 0, 1))
    visualizer.meshcat_cspace.SetTransform(f"/{s1_name}",
                                   RigidTransform(RotationMatrix(),
                                                 s1_3d))
    visualizer.meshcat_cspace.SetLine(f"/edge_{s0_name}_{s1_name}", 
                                np.hstack([s0_3d[:,np.newaxis], s1_3d[:, np.newaxis]]),
                                line_width = 2, rgba = Rgba(0.0, 0.0, 0.0, 1))
plot_edge(s0,s1_safe, s1_name = "s1_safe")
plot_edge(s0,s1_unsafe, s1_name = "s1_unsafe")
plot_edge(limits_s[0],limits_s[1], s0_name = "joint limit low", s1_name = "joint limit high")

[0 0 0]
[-0.25  0.9   0.  ]
[0 0 0]
[1 1 0]
[-0.99920399 -0.99920399  0.        ]
[0.99920399 0.99920399 0.        ]


In [11]:
def make_line_poly(start_point, end_point):
    polys = np.empty(shape=(plant.num_positions(), 1), dtype = object)
    for j in range(plant.num_positions()):
        polys[j] = PolynomialCommon(np.array([start_point[j]-end_point[j], end_point[j]]))
    return polys
safe_line = make_line_poly(s0, s1_safe)
unsafe_line = make_line_poly(s0, s1_unsafe)

In [12]:
cert_options = CspaceFreePath.FindSeparationCertificateGivenPathOptions()
cert_options.terminate_segment_certification_at_failure = False

cert_options.num_threads = -1
cert_options.verbose = False
cert_options.solver_id = MosekSolver.id()
cert_options.solver_options = SolverOptions()
cert_options.terminate_path_certification_at_failure = False


In [13]:

is_safe_safe, safe_cert = cspace_free_path.FindSeparationCertificateGivenPath(safe_line, set(), cert_options)
print(is_safe_safe)
is_safe_unsafe, unsafe_cert = cspace_free_path.FindSeparationCertificateGivenPath(unsafe_line, set(), cert_options)
print(is_safe_unsafe)


[True]
[False]


In [14]:
pair, plane_idx = next(iter(cspace_free_path.map_geometries_to_separating_planes().items()))
pair

(<GeometryId value=22>, <GeometryId value=36>)

In [15]:
for pair, plane_idx in cspace_free_path.map_geometries_to_separating_planes().items():
    print(pair)

(<GeometryId value=22>, <GeometryId value=36>)


In [16]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]

visualizer.meshcat_cspace.Delete("/prm")

def plot_prm(nodes, adjacency_list, width, color = Rgba(0.0, 0.0, 1, 1), prefix = ""):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = np.append(nodes[node_idx, :],0)
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = np.append(nodes[adjacency_list[node_idx][edge_idx], :],0)
            name = f"/{prefix}/prm/rm/line/{plt_idx}"
#             for c in map(int, sqtring):
#                 name += f"/{c}"
            visualizer.meshcat_cspace.SetLine(name,  np.hstack([pos1[:,np.newaxis], pos2[:, np.newaxis]]),
                                 line_width = width, rgba = color)
            plt_idx +=1
            
plotting_fn_handle_good = partial(plot_prm, width = 0.1, prefix = "Good", color = Rgba(0, 0.,0,1))

def collision(pos, col_func_handle):
    return col_func_handle(pos)


prm_col_fn_handle = partial(collision, col_func_handle = visualizer.check_collision_s_by_ik)

visualizer.check_collision_s_by_ik(np.zeros_like(q_star))
prm_col_fn_handle(np.zeros_like(q_star))


0.0

In [17]:
num_points = 100

PRM = prm.PRM( 
            limits_s,
            num_points = num_points,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = plotting_fn_handle_good
            )
# if num_points < 100:
PRM.plot()
tot_num_edges = len(PRM.adjacency_list)* len(PRM.adjacency_list[0])

[PRM] Samples 0
[PRM] Samples 30
[PRM] Samples 60
[PRM] Samples 90
[PRM] Nodes connected: 0
[PRM] Nodes connected: 20
[PRM] Nodes connected: 40
[PRM] Nodes connected: 60
[PRM] Nodes connected: 80


In [18]:
def make_line_polys(PRM):
    endpoint_index_set = set()
    for neighbors in PRM.adjacency_list:
        for n in neighbors[1:]:
            endpoint_index_set.add((neighbors[0], n))
    polys = np.empty(shape=(plant.num_positions(), len(endpoint_index_set)), dtype = object)
    for i, (idx0, idx1) in enumerate(endpoint_index_set):
        s0 = PRM.nodes[idx0]
        s1 = PRM.nodes[idx1]
        for j in range(plant.num_positions()):
            polys[j,i] = PolynomialCommon(np.array([s0[j]-s1[j], s1[j]]))
    return polys

path_safe = make_line_polys(PRM)

In [19]:
t0 = time.time()
edge_safe, certs = cspace_free_path.FindSeparationCertificateGivenPath(path_safe[:,0], set(), cert_options)
t1 = time.time()
print(f"Certification of safe PRM in {t1-t0}s")

Certification of safe PRM in 0.09583425521850586s


In [20]:
edge_safe

[True]

In [21]:
from pydrake.multibody.rational_forward_kinematics import SeparatingPlaneOrderOld, VerificationOptionOld
diagram = visualizer.task_space_diagram
# context = visualizer.diagram_context
# sg_context = scene_graph.GetMyContextFromRoot(context)
# inspector = scene_graph.model_inspector()

# pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

# gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
# get_name_of_gid = lambda gid : inspector.GetName(gid)
# gids.sort(key=get_name_of_gid)

# id_pairs_of_interest = [(gid1, gid2) for i, gid1 in enumerate(gids) for gid2 in gids[i+1:] ]
# print(id_pairs_of_interest)
# visualizer.collision_pairs_of_interest = id_pairs_of_interest
t0 = time.time()
line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                SeparatingPlaneOrderOld.kAffine, 
                                np.zeros(plant.num_positions()),
                                set(), VerificationOptionOld()
                               )
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

Time to construct line certifier = 0.022484302520751953s


In [22]:
safe_line = make_line_poly(s0, s1_safe)
sep_cert_prog = cspace_free_path.MakeIsGeometrySeparableOnPathProgram(pair, safe_line)
print(sep_cert_prog.prog())

Decision variables: plane_var0 plane_var1 plane_var2 plane_var3 plane_var4 plane_var5 plane_var6 plane_var7    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)    Sv(0,0)    Sv(1,0)    Sv(1,1)    Sl(0,0)    Sl(1,0)    Sl(1,1)  

In [23]:
old_prog = line_certifier.MakeCertificationProgram(s0,s1_safe)
print(old_prog)

Decision variables:  plane_var0  plane_var1  plane_var2  plane_var3  plane_var4  plane_var5  plane_var6  plane_var7  plane_var8  plane_var9 plane_var10 plane_var11     Sl(0,0)     Sl(1,0)     Sl(1,1)     Sv(0,0)     Sv(1,0)     Sv(1,1)     Sl(0,0)     Sl(1,0)     Sl(1,1)     Sv(0,0)     Sv(1,0)     Sv(1,1)     Sl(0,0)     Sl(1,0)     Sl(1,1)     Sv(0,0)     Sv(1,0)     Sv(1,1)     Sl(0,0)     Sl(1,0)     Sl(1,1)     Sv(0,0)     Sv(1,0)     Sv(1,1)     Sl(0,0)     Sl(1,0)     Sl(1,1)     Sv(0,0)     Sv(1,0)     Sv(1,1)     Sl(0,0)     Sl(1,0)     Sl(1,1)     Sv(0,0)     Sv(1,0)     Sv(1,1)     Sl(0,0)     Sl(1,0)     Sl(1,1)     Sv(0,0)     Sv(1,0)     Sv(1,1)     Sl(0,0)     Sl(1,0)     Sl(1,1)     Sv(0,0)     Sv(1,0)     Sv(1,1)     Sl(0,0)     Sl(1,0)     Sl(1,1)     Sv(0,0)     Sv(1,0)     Sv(1,1)     Sl(0,0)     Sl(1,0)     Sl(1,1)     Sv(0,0)     Sv(1,0)     Sv(1,1)     Sl(0,0)     Sl(1,0)     Sl(1,1)     Sv(0,0)     Sv(1,0)     Sv(1,1)     Sl(0,0)     Sl(1,0)     Sl(1,1)     Sv(0

In [43]:
line_certifier.get_tuples()[0].get_p()

<Polynomial "(-1 + 1.5 * plane_var6 + 0.5 * plane_var7 - 5 * plane_var8 + plane_var11 - Sv(1,1) + 1.5 * (plane_var0 * s1(0)) + 10 * (plane_var0 * pow(s1(0), 2)) - 1.5 * (plane_var0 * pow(s1(0), 3)) + 10 * (plane_var1 * s1(0) * s1(1)) - 1.5 * (plane_var1 * pow(s1(0), 2) * s1(1)) + 1.5 * (plane_var1 * s1(1)) + 0.5 * (plane_var2 * s1(0)) + 0.5 * (plane_var2 * pow(s1(0), 3)) + 0.5 * (plane_var3 * pow(s1(0), 2) * s1(1)) + 0.5 * (plane_var3 * s1(1)) - 5 * (plane_var4 * s1(0)) + 3 * (plane_var4 * pow(s1(0), 2)) + 5 * (plane_var4 * pow(s1(0), 3)) + 3 * (plane_var5 * s1(0) * s1(1)) + 5 * (plane_var5 * pow(s1(0), 2) * s1(1)) - 5 * (plane_var5 * s1(1)) + 10 * (plane_var6 * s1(0)) - 1.5 * (plane_var6 * pow(s1(0), 2)) + 0.5 * (plane_var7 * pow(s1(0), 2)) + 3 * (plane_var8 * s1(0)) + 5 * (plane_var8 * pow(s1(0), 2)) + (plane_var9 * s1(0)) + (plane_var9 * pow(s1(0), 3)) + (plane_var10 * pow(s1(0), 2) * s1(1)) + (plane_var10 * s1(1)) + (plane_var11 * pow(s1(0), 2)) - pow(s1(0), 2))*1 + ( - Sl(1,1) - 2

In [41]:
tmp = cspace_free_path.plane_geometries_on_path()[0].positive_side_conditions + \
    cspace_free_path.plane_geometries_on_path()[0].negative_side_conditions
tmp[0].get_p()

<Polynomial "(-1 + 2.5 * plane_var1 + 0.5 * plane_var3 + 5 * plane_var5 + plane_var7 - Sv(1,1) - 10 * (s_0(mu_1)_coeff * plane_var1) + 5 * (s_0(mu_1)_coeff * plane_var5) - 2.5 * (pow(s_0(mu_1)_coeff, 2) * plane_var1) + 0.5 * (pow(s_0(mu_1)_coeff, 2) * plane_var3) - 5 * (pow(s_0(mu_1)_coeff, 2) * plane_var5) + (pow(s_0(mu_1)_coeff, 2) * plane_var7) - pow(s_0(mu_1)_coeff, 2))*1 + (2.5 * plane_var0 + 0.5 * plane_var2 + 5 * plane_var4 + plane_var6 - Sl(1,1) - 2 * Sv(1,0) + Sv(1,1) - 2 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff) - 5 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff * plane_var1) + (s_0(mu_0)_coeff * s_0(mu_1)_coeff * plane_var3) - 10 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff * plane_var5) + 2 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff * plane_var7) - 10 * (s_0(mu_0)_coeff * plane_var1) + 5 * (s_0(mu_0)_coeff * plane_var5) - 10 * (s_0(mu_1)_coeff * plane_var0) + 5 * (s_0(mu_1)_coeff * plane_var4) - 2.5 * (pow(s_0(mu_1)_coeff, 2) * plane_var0) + 0.5 * (pow(s_0(mu_1)_coeff, 2) * plane_var2) - 5 * (pow(s_0(